In [1]:
# Time to figure out what's going on and GET ORGANIZED
""" Sets of files: 
    -rules_based_class_tags.py and rule_tag_main.py
        * These run an experiment using the TAGS as rules
    -rules_based_class_changes.py and rules_main.py
        * These run an experiment using CHANGES (file paths) as rules
    -rules_based_class.py and rules_exploration.py
        * These run an ORIGINAL experiment with no Praxi involved
"""

no_vers = '/home/ubuntu/praxi/demo/demo_changesets/sl_train'

changeset_dir = '/home/ubuntu/praxi/version_detection/changeset_compare/more_ex_vl'
tagset_dir = '/home/ubuntu/praxi/version_detection/tagset_compare/more_ex_vl'

nfolds = 10

outdir = '/home/ubuntu/praxi/rules_based/results'

resfile_name = 'rules_exp.p'

In [2]:
print(resfile_name)

rules_exp.p


In [3]:
# ORIGINAL RULE CLASS
from collections import OrderedDict
from orderedset import OrderedSet
import logging


class RuleBased:
    """ scikit-style wrapper """
    def __init__(self, threshold=0.5, num_rules=1, max_index=20, string_rules=False,
                 unknown_label='???', filter_method='vlad'):
        self.threshold = threshold
        self.max_index = max_index
        self.string_rules = string_rules # bool
        self.unknown_label = unknown_label 
        self.filter_method = filter_method # str
        self.num_rules = num_rules

    def fit(self, X, y, csids=None):
        if self.filter_method == 'vlad':
            label_to_tokens = self._transform_anthony_intersection(X, y)
        elif self.filter_method == 'intersect':
            label_to_tokens = self._intersect_packages(X, y)
        elif self.filter_method == 'take_max':
            label_to_tokens = self._transform_anthony_intersection(X, y, take_max=True)
        else:
            raise ValueError("Unknown filter method %s" % self.filter_method)
 
        # Get the inverse map
        token_to_labels = get_token_to_labels(label_to_tokens)
        # Get the map from labels to categorized tokens
        label_to_token_groups = get_label_to_token_groups(token_to_labels)
        # Find duplicates
        duplicates = get_duplicates(label_to_tokens, token_to_labels,
                                    label_to_token_groups)
        print("Duplicates:", duplicates)
        # Filter out duplicates from the corpus
        for k in label_to_tokens:
            if k in duplicates:
                del label_to_tokens[k]
        # Again get the inverse map
        token_to_labels = get_token_to_labels(label_to_tokens)
        # Again get the map from labels to categorized tokens
        label_to_token_groups = get_label_to_token_groups(token_to_labels)
        print("Label-to-token groups:", label_to_token_groups)
        # Generate rules for all labels
        rules = get_rules(label_to_tokens, token_to_labels,
                          label_to_token_groups, limit=self.num_rules,
                          max_index=self.max_index,
                          string_rules=self.string_rules)

        # Filter out rules for labels that are not in Anthony's data
        self.rules = OrderedDict()
        for k, v in rules.items():
            if k in y:
                self.rules[k] = v
        logging.info('Finished rule generation')

    def predict(self, X, csids=None, ntags=None):
        if ntags is None:
            unravel_result = True
            ntags = [1 for _ in len(X)]
        else:
            unravel_result = False
        predictions = []
        for n_preds, changes in zip(ntags, X):
            cur_predictions = {}
            for label_tested, label_rules in self.rules.items():
                n_rules_satisfied = 0
                n_rules = len(label_rules)
                if n_rules == 0:
                    logging.info("%s has no rules", label_tested)
                    continue
                for rule in label_rules:
                    rule_satisfied = True
                    for triplet in rule:
                        token = triplet[0]
                        inside = (triplet[1] != 'outside vs')
                        if inside == (len([v for v in changes
                                           if token == v[-len(token):]]) == 0):
                            rule_satisfied = False
                            break
                    if rule_satisfied:
                        n_rules_satisfied += 1
                cur_predictions[label_tested] = n_rules_satisfied / n_rules
            result = []
            for _ in range(n_preds):
                if (not cur_predictions) or (
                        max(cur_predictions.values()) == 0) or (
                        max(cur_predictions.values()) < self.threshold and
                        n_preds == 1):
                    result.append(self.unknown_label)
                else:
                    best_pred = max(cur_predictions, key=cur_predictions.get)
                    result.append(best_pred)
                    del cur_predictions[best_pred]
            if unravel_result:
                predictions.append(result[0])
            else:
                predictions.append(result)
        logging.info('Finished rule checking')
        return predictions

    def top_k_tags(self, X, ntags):
        return self.predict(X, ntags=ntags)

    def get_args(self):
        return 'threshold: {}, max_index: {}, num_rules: {}'.format(
            self.threshold, self.max_index, self.num_rules)

    def _intersect_packages(self, changesets, labels):
        """Keep only files present in every instance of a package."""
        res = OrderedDict()
        for data, label in zip(changesets, labels):
            if label in res:
                res[label] &= OrderedSet(data)
                if not len(res[label]):
                    logging.warning("No common files for package %s" % label)
            else:
                res[label] = OrderedSet(data)
        return res

    def _transform_anthony_intersection(self, changesets, labels, take_max=False):
        res = OrderedDict()
        # res[package_name][file_name] = no. of occurances
        for data, label in zip(changesets, labels):
            for token in data:
                if label not in res:
                    res[label] = OrderedDict()
                if token not in res[label]:
                    res[label][token] = 1
                else:
                    res[label][token] += 1
        newres = OrderedDict()
        # newres[package_name] = set(file_names) s.t.
        # freq. of file satisfies mystery_vlad_condition
        for label in res:
            newres[label] = OrderedSet()
            maxval = max(res[label].values())
            for token in sorted(res[label], key=res[label].get, reverse=True):
                mystery_vlad_condition = (
                    (res[label][token] != maxval
                        and len(newres[label]) > 50) or
                    (res[label][token] < 0.94 * maxval
                        and len(newres[label]) >= 40) or
                    (res[label][token] < 0.88 * maxval
                        and len(newres[label]) >= 26) or
                    (res[label][token] < 0.8 * maxval
                        and len(newres[label]) >= 16) or
                    (res[label][token] < 0.7 * maxval
                        and len(newres[label]) >= 10) or
                    (res[label][token] < 0.6 * maxval
                        and len(newres[label]) >= 8) or
                    (res[label][token] < 0.5 * maxval
                        and len(newres[label]) >= 6)
                )
                if take_max:
                    if res[label][token] != maxval:
                        break
                else:
                    if mystery_vlad_condition:
                        break
                newres[label].add(token)
        return newres


def get_token_to_labels(label_to_tokens):
    """
    Returns the inverse map: a dictionary from tokens to sets of labels.
    """
    token_to_labels = OrderedDict()
    for label in label_to_tokens:
        for token in label_to_tokens[label]:
            if token not in token_to_labels:
                token_to_labels[token] = OrderedSet()
            token_to_labels[token].add(label)
    return token_to_labels


def get_label_to_token_groups(token_to_labels):
    """
    Returns a categorized corpus. It's a dictionary from labels to groups
    of tokens. These groups are indexed with natural numbers. Index of a
    group shows in how many labels each token from this group is present.
    """
    label_to_token_groups = OrderedDict()
    for token in token_to_labels:
        for label in token_to_labels[token]:
            index = len(token_to_labels[token])
            if label not in label_to_token_groups:
                label_to_token_groups[label] = OrderedDict()
            if index not in label_to_token_groups[label]:
                label_to_token_groups[label][index] = OrderedSet()
            label_to_token_groups[label][index].add(token)
    return label_to_token_groups


def get_duplicates(label_to_tokens, token_to_labels, label_to_token_groups):
    """
    Returns labels, not all, that have sets of tokens identical to other
    labels. From each group of identical labels one label goes to
    representatives. All the other labels from each group go to <duplicates>.
    """
    duplicates = OrderedSet()
    for label in sorted(label_to_tokens.keys()):
        if label in duplicates:
            continue
        first_index = sorted(label_to_token_groups[label].keys())[0]
        first_token = list(label_to_token_groups[label][first_index])[0]
        potential_duplicates = token_to_labels[first_token]
        for other_label in sorted(list(potential_duplicates)):
            if (other_label <= label) or (other_label in duplicates):
                continue
            if label_to_tokens[label] == label_to_tokens[other_label]:
                duplicates.add(other_label)
                logging.info(
                    'Duplicates: {0} = {1}'.format(label, other_label))
    return duplicates


def get_rules_per_label(label, label_to_tokens, token_to_labels,
                        label_to_token_groups, limit=1, max_index=0,
                        string_rules=False):
    """
    Generates rules, at most <limit>, for a specified <label>.
    Each rule is a list of <index> triplets.
    Each rules includes exactly one triplet
    of the format:
        (*) (<token>, 'unique to', <index>)
    It means that <token> appears exactly in <index> different labels including
    <label>. All these labels, except <label>, are listed exactly once in other
    triplets as <other_label>. A triplet of this format always goes first.
    Other triplets have the formats:
        (1) (<token>, 'inside vs', <other_label>) or
        (2) (<token>, 'outside vs', <other_label>)
    It means that <token> distinguishes <label> from <other_label>. Format (1)
    means that <token> is in <label> but not in <other_label>. Format (2) means
    that <token> is in <other_label> but not in <label>.
    Rules are ordered in a list according to <index>. There could be less rules
    than <limit>. Across all rules each token can appear only once in a triplet
    of format (*) and only once in a triplet of format (1) or (2). This
    guarantees that changes to one token will affect at most two rules. It's
    also guaranteed that rules have the smallest possible <indeces> under the
    requirement given above.
    """
    assert (label in label_to_token_groups)
    rules = []
    used_tokens = OrderedSet()
    for index in sorted(label_to_token_groups[label].keys()):
        if index > max_index and max_index > 0:
            break
        for token in label_to_token_groups[label][index]:
            if token in used_tokens:
                continue
            rule = []
            if string_rules:
                rule.append(token + ' unique to ' + str(index))
            else:
                rule.append((token, 'unique to', str(index)))
            for other_label in token_to_labels[token]:
                if other_label == label:
                    continue
                plus_diff = label_to_tokens[label] - \
                    label_to_tokens[other_label]
                minus_diff = label_to_tokens[other_label] - \
                    label_to_tokens[label]
                assert (len(plus_diff) + len(minus_diff)) > 0
                plus_diff -= used_tokens
                minus_diff -= used_tokens
                if len(plus_diff) > 0:
                    if string_rules:
                        rule.append(
                            list(plus_diff)[0] + ' inside vs ' + other_label)
                    else:
                        rule.append(
                            (list(plus_diff)[0], 'inside vs', other_label))
                elif len(minus_diff) > 0:
                    if string_rules:
                        rule.append(
                            list(minus_diff)[0] + ' outside vs ' + other_label)
                    else:
                        rule.append(
                            (list(minus_diff)[0], 'outside vs', other_label))
                else:
                    break
            if len(rule) < index:
                continue
            rules.append(rule)
            for triplet in rule:
                used_tokens.add(triplet[0])
            if len(rules) >= limit:
                return rules
    return rules


def get_rules(label_to_tokens, token_to_labels, label_to_token_groups,
              limit=1, max_index=5, string_rules=False):
    """
    Generates a dictionary from labels to sets of rules.
    See description of <get_rules_per_label> for more details.
    """
    rules = OrderedDict()
    for label in label_to_token_groups:
        rules[label] = get_rules_per_label(
            label, label_to_tokens, token_to_labels,
            label_to_token_groups, limit, max_index, string_rules)
    return rules



In [4]:
# Partition names into folds
import logging
import logging.config

import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import random
#import tempfile
import time
import yaml
import pickle
import copy
import argparse

from sklearn.base import BaseEstimator
from tqdm import tqdm

import numpy as np
from numpy import savetxt
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

#from hybrid_tags import Hybrid
#from rule_orig_class import RuleBased
from collections import OrderedDict
from orderedset import OrderedSet


def fold_partitioning(ts_names, n=3):
    # partition tagsets into folds for cross validation
    folds = [[] for _ in range(n)]

    just_progs = []
    for name in ts_names:
        ts_name_comps = name.split('.')
        just_progs.append(ts_name_comps[0])

    prog_set = set(just_progs)
    unique_progs = (list(prog_set))

    prog_partition = [[] for _ in range(len(unique_progs))]

    for name in ts_names:
        name_comps = name.split('.')
        just_pname = name_comps[0]
        for i, prog in enumerate(unique_progs):
            if just_pname == prog:
                prog_partition[i].append(name)

    for ts_names in prog_partition:
        for idx, name in enumerate(ts_names):
            folds[idx % n].append(name)

    return folds

clf = RuleBased(num_rules=6)

resfile = open(resfile_name, 'wb')
results = []

cs_names = [f for f in listdir(no_vers) if (isfile(join(no_vers, f))and f[-4:]=='yaml')]

random.shuffle(cs_names)
#logging.info("Partitioning into %d folds", nfolds)
folds = fold_partitioning(cs_names, n=nfolds)
#logging.info("Starting cross validation folds: ")




In [5]:
cs_path = no_vers

def get_set(dat_path): # combine with parse_ts
    # Argument: - complete path to a tagset .yaml file
    # Returns:  - tagset dictionary contained in file (tags, labels)
    with open(dat_path, 'r') as stream:
        data_loaded = yaml.load(stream)
    return data_loaded

def parse_cs(changeset_names, cs_dir):
    # Arguments: - tagset_names: a list of names of tagsets
    #            - ts_dir: the directory in which they are located
    # Returns: - tags: list of lists-- tags for each tagset name
    #          - labels: application name corresponding to each tagset
    #tags = []
    #labels = []
    changesets = []
    for cs_name in tqdm(changeset_names):
            cs_path = cs_dir + '/' + cs_name
            changeset = get_set(cs_path)
            changesets.append(changeset)
    return changesets


idx = 0
test_names = folds[idx]
train_names_list = list(range(len(folds)))
train_names_list.remove(idx)
#logging.info("Training folds: %s", str(train_idx_list))
train_names = []
for i in train_names_list:
    train_names += folds[i]

train_dics = parse_cs(train_names, cs_path)
test_dics = parse_cs(test_names,  cs_path)

X_train = []
y_train = []
X_test = []
y_test = []

for tr_d in train_dics:
    X_train.append(tr_d['changes'])
    y_train.append(tr_d['label'])

for ts_d in test_dics:
    X_test.append(ts_d['changes'])
    y_test.append(ts_d['label'])


100%|██████████| 100/100 [00:19<00:00,  1.56it/s]


In [6]:
print(X_test[0])

['000 /var/cache/debconf/config.dat-new', '644 /var/cache/debconf/config.dat-old', '644 /var/cache/debconf/config.dat', '000 /var/lib/apt/extended_states.Mb2eVq', '644 /var/lib/apt/extended_states', '000 /usr/share/lintian/overrides/libapr1.dpkg-new', '644 /usr/share/doc/libapr1/copyright', '644 /usr/share/doc/libapr1/changelog.Debian.gz', '000 /usr/lib/x86_64-linux-gnu/libapr-1.so.0.5.2.dpkg-new', '000 /usr/lib/x86_64-linux-gnu/libapr-1.so.0.dpkg-new', '644 /usr/share/lintian/overrides/libapr1', '644 /usr/lib/x86_64-linux-gnu/libapr-1.so.0.5.2', '644 /usr/lib/x86_64-linux-gnu/libapr-1.so.0', '000 /var/lib/dpkg/info/libapr1:amd64.list-new', '644 /var/lib/dpkg/info/libapr1:amd64.list', '755 /var/lib/dpkg/info/libapr1:amd64.postinst', '644 /var/lib/dpkg/info/libapr1:amd64.md5sums', '644 /var/lib/dpkg/info/libapr1:amd64.shlibs', '644 /var/lib/dpkg/info/libapr1:amd64.symbols', '755 /var/lib/dpkg/info/libapr1:amd64.postrm', '000 /usr/lib/x86_64-linux-gnu/libaprutil-1.so.0.5.4.dpkg-new', '64

In [6]:
#clf.fit(X_train, y_train)

def transform_anthony_intersection(changesets, labels, take_max=False):
    res = OrderedDict()
    # res[package_name][file_name] = no. of occurances
    for data, label in zip(changesets, labels):
        for token in data:
            if label not in res:
                res[label] = OrderedDict()
            if token not in res[label]:
                res[label][token] = 1
            else:
                res[label][token] += 1
    newres = OrderedDict()

    # newres[package_name] = set(file_names) s.t.
    # freq. of file satisfies mystery_vlad_condition
    for label in res:
        #print(res[label])
        #input("Enter to continue...")
        newres[label] = OrderedSet()
        maxval = max(res[label].values())
        print(maxval)
        for token in sorted(res[label], key=res[label].get, reverse=True):
            #print(res[label][token])
            #print("res[label][token]: ", res[label][token], "Number of tokens for this label: ", len(newres[label]))
            #input("Enter to continue...")
            mystery_vlad_condition = (
                (res[label][token] != maxval
                    and len(newres[label]) > 50) or
                (res[label][token] < 0.94 * maxval
                    and len(newres[label]) >= 40) or
                (res[label][token] < 0.88 * maxval
                    and len(newres[label]) >= 26) or
                (res[label][token] < 0.8 * maxval
                    and len(newres[label]) >= 16) or
                (res[label][token] < 0.7 * maxval
                    and len(newres[label]) >= 10) or
                (res[label][token] < 0.6 * maxval
                    and len(newres[label]) >= 8) or
                (res[label][token] < 0.5 * maxval
                    and len(newres[label]) >= 6)
            )
            #print(mystery_vlad_condition)
            #input("enter to continue...")
            if take_max:
                if res[label][token] != maxval:
                    break
            else:
                if mystery_vlad_condition:
                    print(res[label][token])
                    break
            newres[label].add(token)
    return newres


label_to_tokens = transform_anthony_intersection(X_train, y_train)


54
53
54
36
54
51
54
53
54
36
54
36
54
36
54
52
54
36
54
45
54
18
54
36
54
45
54
36
54
36
54
36
54
45
54
36
54
51
24
23
54
39
54
45
54
36
27
21
54
36
54
53
54
36
54
51
54
24
54
50
54
48
54
36
54
36
54
36
54
36
54
52
54
24
54
21
54
36
54
36
54
36
54
48
54
36
54
36
54
36
54
36
54
53
54
48
54
36
54
36


In [7]:
print(len(label_to_tokens['zsh']))
print(label_to_tokens['zsh'])

51
OrderedSet(['000 /var/lib/dpkg/tmp.ci/control', '000 /var/lib/dpkg/tmp.ci/prerm', '000 /var/lib/dpkg/tmp.ci/md5sums', '000 /var/lib/dpkg/tmp.ci/preinst', '000 /var/lib/dpkg/tmp.ci/postinst', '000 /var/lib/dpkg/tmp.ci/postrm', '000 /bin/zsh5.dpkg-new', '000 /usr/share/bug/zsh.dpkg-new', '000 /usr/share/lintian/overrides/zsh.dpkg-new', '000 /var/lib/dpkg/info/zsh.list-new', '000 /etc/shells.tmp', '000 /var/lib/dpkg/alternatives/zsh.dpkg-tmp', '000 /var/lib/dpkg/alternatives/rzsh.dpkg-tmp', '000 /var/lib/dpkg/updates/tmp.i', '000 /var/lib/dpkg/status-new', '000 /var/lib/dpkg/tmp.ci/conffiles', '000 /etc/zsh/zshenv.dpkg-new', '000 /etc/zsh/zlogin.dpkg-new', '000 /etc/zsh/zshrc.dpkg-new', '000 /etc/zsh/zprofile.dpkg-new', '000 /etc/zsh/newuser.zshrc.recommended.dpkg-new', '000 /etc/zsh/zlogout.dpkg-new', '000 /usr/share/bug/zsh-common.dpkg-new', '000 /usr/share/lintian/overrides/zsh-common.dpkg-new', '000 /usr/share/menu/zsh-common.dpkg-new', '000 /usr/share/man/man1/zshexpn.1.gz.dpkg-ne

In [8]:
def get_token_to_labels(label_to_tokens):
    """
    Returns the inverse map: a dictionary from tokens to sets of labels.
    """
    token_to_labels = OrderedDict()
    for label in label_to_tokens:
        for token in label_to_tokens[label]:
            if token not in token_to_labels:
                token_to_labels[token] = OrderedSet()
            token_to_labels[token].add(label)
    return token_to_labels

token_to_labels = get_token_to_labels(label_to_tokens)
print(token_to_labels)

OrderedDict([('000 /var/lib/dpkg/tmp.ci/prerm', OrderedSet(['dovecot-core', 'lynx', 'freeradius', 'sysstat', 'smartmontools', 'wordpress', 'arpwatch', 'ksh', 'gitweb', 'quota', 'vsftpd', 'watchdog', 'units', 'nmap', 'quagga', 'zsh', 'bacula-client', 'qemu-kvm', 'iotop', 'dstat', 'memcached', 'bind9', 'traceroute', 'mutt'])), ('000 /var/lib/dpkg/tmp.ci/postinst', OrderedSet(['dovecot-core', 'lynx', 'freeradius', 'sysstat', 'docker', 'smartmontools', 'wordpress', 'mgetty-voice', 'arpwatch', 'ksh', 'gitweb', 'quota', 'vsftpd', 'watchdog', 'autotrace', 'units', 'nmap', 'quagga', 'zsh', 'bacula-client', 'qemu-kvm', 'iotop', 'mtr', 'squid', 'dstat', 'memcached', 'subversion', 'bind9', 'sane', 'traceroute', 'crda', 'mutt'])), ('000 /var/lib/dpkg/tmp.ci/conffiles', OrderedSet(['dovecot-core', 'lynx', 'logwatch', 'freeradius', 'iptraf-ng', 'sysstat', 'smartmontools', 'lftp', 'wordpress', 'mgetty-voice', 'arpwatch', 'ksh', 'gitweb', 'quota', 'vsftpd', 'watchdog', 'quagga', 'zsh', 'bacula-client'

In [9]:
print(token_to_labels['000 /var/lib/dpkg/tmp.ci/control'])

OrderedSet(['dovecot-core', 'lynx', 'logwatch', 'freeradius', 'rdist', 'cobertura', 'iptraf-ng', 'sysstat', 'docker', 'smartmontools', 'tree', 'lftp', 'wordpress', 'mgetty-voice', 'arpwatch', 'ksh', 'mariadb-server', 'gitweb', 'quota', 'vsftpd', 'watchdog', 'autotrace', 'units', 'nmap', 'pax', 'quagga', 'dcraw', 'zsh', 'lshw', 'bacula-client', 'qemu-kvm', 'iotop', 'supermin', 'tshark', 'mtr', 'squid', 'finger', 'emacs-nox', 'dstat', 'memcached', 'subversion', 'bind9', 'sane', 'traceroute', 'powertop', 'crda', 'mutt', 'marisa'])


In [10]:
def get_label_to_token_groups(token_to_labels):
    """
    Returns a categorized corpus. It's a dictionary from labels to groups
    of tokens. These groups are indexed with natural numbers. Index of a
    group shows in how many labels each token from this group is present.
    """
    label_to_token_groups = OrderedDict()
    for token in token_to_labels:
        for label in token_to_labels[token]:
            index = len(token_to_labels[token])
            if label not in label_to_token_groups:
                label_to_token_groups[label] = OrderedDict()
            if index not in label_to_token_groups[label]:
                label_to_token_groups[label][index] = OrderedSet()
            label_to_token_groups[label][index].add(token)
    return label_to_token_groups

label_to_token_groups = get_label_to_token_groups(token_to_labels)
# Find duplicates
print(label_to_token_groups['dcraw'])

OrderedDict([(48, OrderedSet(['000 /var/lib/dpkg/tmp.ci/md5sums', '000 /var/lib/dpkg/tmp.ci/control', '000 /var/lib/dpkg/updates/tmp.i', '000 /var/lib/dpkg/status-new'])), (1, OrderedSet(['000 /usr/bin/dcparse.dpkg-new', '000 /usr/bin/dcfujigreen.dpkg-new', '000 /usr/bin/dccleancrw.dpkg-new', '000 /usr/bin/dcraw.dpkg-new', '000 /usr/bin/dcfujiturn16.dpkg-new', '000 /usr/bin/dcfujiturn.dpkg-new', '000 /usr/share/man/man1/dcraw.1.gz.dpkg-new', '000 /usr/share/man/man1/dcfujiturn.1.gz.dpkg-new', '000 /usr/share/man/man1/dcparse.1.gz.dpkg-new', '000 /usr/share/man/man1/dcfujigreen.1.gz.dpkg-new', '000 /usr/share/man/man1/dccleancrw.1.gz.dpkg-new', '000 /usr/share/man/man1/dcfujiturn16.1.gz.dpkg-new', '000 /var/lib/dpkg/info/dcraw.list-new']))])


In [11]:

def get_duplicates(label_to_tokens, token_to_labels, label_to_token_groups):
    """
    Returns labels, not all, that have sets of tokens identical to other
    labels. From each group of identical labels one label goes to
    representatives. All the other labels from each group go to <duplicates>.
    """
    duplicates = OrderedSet()
    for label in sorted(label_to_tokens.keys()):
        if label in duplicates:
            continue
        first_index = sorted(label_to_token_groups[label].keys())[0]
        first_token = list(label_to_token_groups[label][first_index])[0]
        potential_duplicates = token_to_labels[first_token]
        for other_label in sorted(list(potential_duplicates)):
            if (other_label <= label) or (other_label in duplicates):
                continue
            if label_to_tokens[label] == label_to_tokens[other_label]:
                duplicates.add(other_label)
                logging.info(
                    'Duplicates: {0} = {1}'.format(label, other_label))
    return duplicates

duplicates = get_duplicates(label_to_tokens, token_to_labels, label_to_token_groups)

In [12]:
def get_rules_per_label(label, label_to_tokens, token_to_labels,
                        label_to_token_groups, limit=1, max_index=0,
                        string_rules=False):
    """
    Generates rules, at most <limit>, for a specified <label>.
    Each rule is a list of <index> triplets.
    Each rules includes exactly one triplet
    of the format:
        (*) (<token>, 'unique to', <index>)
    It means that <token> appears exactly in <index> different labels including
    <label>. All these labels, except <label>, are listed exactly once in other
    triplets as <other_label>. A triplet of this format always goes first.
    Other triplets have the formats:
        (1) (<token>, 'inside vs', <other_label>) or
        (2) (<token>, 'outside vs', <other_label>)
    It means that <token> distinguishes <label> from <other_label>. Format (1)
    means that <token> is in <label> but not in <other_label>. Format (2) means
    that <token> is in <other_label> but not in <label>.
    Rules are ordered in a list according to <index>. There could be less rules
    than <limit>. Across all rules each token can appear only once in a triplet
    of format (*) and only once in a triplet of format (1) or (2). This
    guarantees that changes to one token will affect at most two rules. It's
    also guaranteed that rules have the smallest possible <indeces> under the
    requirement given above.
    """
    assert (label in label_to_token_groups)
    rules = []
    used_tokens = OrderedSet()
    for index in sorted(label_to_token_groups[label].keys()):
        if index > max_index and max_index > 0:
            break
        for token in label_to_token_groups[label][index]:
            if token in used_tokens:
                continue
            rule = []
            if string_rules:
                rule.append(token + ' unique to ' + str(index))
            else:
                rule.append((token, 'unique to', str(index)))
            for other_label in token_to_labels[token]:
                if other_label == label:
                    continue
                plus_diff = label_to_tokens[label] - \
                    label_to_tokens[other_label]
                minus_diff = label_to_tokens[other_label] - \
                    label_to_tokens[label]
                assert (len(plus_diff) + len(minus_diff)) > 0
                plus_diff -= used_tokens
                minus_diff -= used_tokens
                if len(plus_diff) > 0:
                    if string_rules:
                        rule.append(
                            list(plus_diff)[0] + ' inside vs ' + other_label)
                    else:
                        rule.append(
                            (list(plus_diff)[0], 'inside vs', other_label))
                elif len(minus_diff) > 0:
                    if string_rules:
                        rule.append(
                            list(minus_diff)[0] + ' outside vs ' + other_label)
                    else:
                        rule.append(
                            (list(minus_diff)[0], 'outside vs', other_label))
                else:
                    break
            if len(rule) < index:
                continue
            rules.append(rule)
            for triplet in rule:
                used_tokens.add(triplet[0])
            if len(rules) >= limit:
                return rules
    return rules


def get_rules(label_to_tokens, token_to_labels, label_to_token_groups,
              limit=1, max_index=5, string_rules=False):
    """
    Generates a dictionary from labels to sets of rules.
    See description of <get_rules_per_label> for more details.
    """
    rules = OrderedDict()
    for label in label_to_token_groups:
        rules[label] = get_rules_per_label(
            label, label_to_tokens, token_to_labels,
            label_to_token_groups, limit, max_index, string_rules)
        print(label)
        print(rules[label])
        input("Enter to continue...")
    return rules

rules = get_rules(label_to_tokens, token_to_labels,
                  label_to_token_groups, limit=6,
                  max_index=20,
                  string_rules=False)

dovecot-core
[[('000 /lib/systemd/system/dovecot.socket.dpkg-new', 'unique to', '1')], [('000 /lib/systemd/system/dovecot.service.dpkg-new', 'unique to', '1')], [('000 /usr/share/lintian/overrides/dovecot-core.dpkg-new', 'unique to', '1')], [('000 /usr/share/apport/package-hooks/dovecot-core.py.dpkg-new', 'unique to', '1')], [('000 /usr/share/man/man7/pigeonhole.7.gz.dpkg-new', 'unique to', '1')], [('000 /usr/share/man/man7/doveadm-search-query.7.gz.dpkg-new', 'unique to', '1')]]
Enter to continue...
lynx
[[('000 /usr/bin/lynx.dpkg-new', 'unique to', '1')], [('000 /usr/share/lintian/overrides/lynx.dpkg-new', 'unique to', '1')], [('000 /usr/share/menu/lynx.dpkg-new', 'unique to', '1')], [('000 /var/lib/dpkg/info/lynx.list-new', 'unique to', '1')], [('000 /var/lib/dpkg/alternatives/www-browser.dpkg-tmp', 'unique to', '1')], [('000 /etc/lynx/lynx.lss.dpkg-new', 'unique to', '1')]]
Enter to continue...
freeradius
[[('000 /etc/logrotate.d/freeradius.dpkg-new', 'unique to', '1')], [('000 /et

Enter to continue...
qemu-kvm
[[('000 /usr/share/man/man1/kvm.1.gz.dpkg-new', 'unique to', '1')], [('000 /usr/bin/kvm.dpkg-new', 'unique to', '1')], [('000 /usr/bin/kvm-spice.dpkg-new', 'unique to', '1')], [('000 /usr/bin/qemu-system-x86_64-spice.dpkg-new', 'unique to', '1')], [('000 /var/lib/dpkg/info/qemu-kvm.list-new', 'unique to', '1')], [('000 /usr/lib/x86_64-linux-gnu/libiscsi.so.2.1.12.dpkg-new', 'unique to', '1')]]
Enter to continue...
iotop
[[('000 /usr/sbin/iotop.dpkg-new', 'unique to', '1')], [('000 /usr/share/man/man8/iotop.8.gz.dpkg-new', 'unique to', '1')], [('000 /usr/share/pyshared/iotop-0.6.egg-info.dpkg-new', 'unique to', '1')], [('000 /usr/lib/python2.7/dist-packages/iotop-0.6.egg-info.dpkg-new', 'unique to', '1')], [('000 /var/lib/dpkg/info/iotop.list-new', 'unique to', '1')]]
Enter to continue...
Enter to continue...
dstat
[[('000 /usr/bin/dstat.dpkg-new', 'unique to', '1')], [('000 /usr/share/python/runtime.d/dstat.rtupdate.dpkg-new', 'unique to', '1')], [('000 /u

Enter to continue...
lftp
[[('000 /etc/lftp.conf.dpkg-new', 'unique to', '1')], [('000 /usr/bin/lftpget.dpkg-new', 'unique to', '1')], [('000 /usr/bin/lftp.dpkg-new', 'unique to', '1')], [('000 /usr/share/man/man1/lftp.1.gz.dpkg-new', 'unique to', '1')], [('000 /usr/share/man/man1/lftpget.1.gz.dpkg-new', 'unique to', '1')], [('000 /usr/share/man/man5/lftp.conf.5.gz.dpkg-new', 'unique to', '1')]]
Enter to continue...
rdist
[[('000 /usr/share/man/man1/rdist.1.gz.dpkg-new', 'unique to', '1')], [('000 /usr/share/man/man1/rdistd.1.gz.dpkg-new', 'unique to', '1')], [('000 /usr/bin/rdist.dpkg-new', 'unique to', '1')], [('000 /usr/bin/rdistd.dpkg-new', 'unique to', '1')], [('000 /var/lib/dpkg/info/rdist.list-new', 'unique to', '1')], [('644 /var/lib/dpkg/status-old', 'unique to', '8'), ('000 /usr/bin/tree.dpkg-new', 'outside vs', 'tree'), ('000 /usr/bin/lshw.dpkg-new', 'outside vs', 'lshw'), ('000 /usr/share/lintian/overrides/supermin.dpkg-new', 'outside vs', 'supermin'), ('000 /usr/bin/tshark

Enter to continue...
marisa
[[('000 /usr/lib/x86_64-linux-gnu/libmarisa.so.0.0.0.dpkg-new', 'unique to', '1')], [('000 /usr/lib/x86_64-linux-gnu/libmarisa.so.0.dpkg-new', 'unique to', '1')], [('000 /var/lib/dpkg/info/libmarisa0:amd64.list-new', 'unique to', '1')], [('000 /usr/bin/marisa-predictive-search.dpkg-new', 'unique to', '1')], [('000 /usr/bin/marisa-benchmark.dpkg-new', 'unique to', '1')], [('000 /usr/bin/marisa-build.dpkg-new', 'unique to', '1')]]
Enter to continue...
lighttpd
[[('000 /var/work/conftest.file', 'unique to', '1')], [('000 /var/work/conftest.make', 'unique to', '1')], [('000 /var/work/conftest.tar', 'unique to', '1')], [('000 /var/work/conftest.dir/file', 'unique to', '1')], [('000 /var/work/conftest2.o', 'unique to', '1')], [('000 /var/work/confinc', 'unique to', '1')]]
Enter to continue...
django
[[('755 /usr/work/django/bin/python', 'unique to', '1')], [('000 /usr/work/django/lib/python3.5/site-packages/setuptools-38.2.4.dist-info/INSTALLER.pip', 'unique to', 

In [ ]:
if ntags is None:
    unravel_result = True
    ntags = [1 for _ in range(len(X))]
else:
    unravel_result = False
predictions = []
for n_preds, changes in zip(ntags, X):
    cur_predictions = {}
    for label_tested, label_rules in self.rules.items():
        n_rules_satisfied = 0
        n_rules = len(label_rules)
        if n_rules == 0:
            logging.info("%s has no rules", label_tested)
            continue
        for rule in label_rules:
            rule_satisfied = True
            for triplet in rule:
                token = triplet[0]
                inside = (triplet[1] != 'outside vs')
                if inside == (len([v for v in changes
                                   if token == v[-len(token):]]) == 0):
                    rule_satisfied = False
                    break
            if rule_satisfied:
                n_rules_satisfied += 1
        cur_predictions[label_tested] = n_rules_satisfied / n_rules
    result = []
    for _ in range(n_preds):
        if (not cur_predictions) or (
                max(cur_predictions.values()) == 0) or (
                max(cur_predictions.values()) < self.threshold and
                n_preds == 1):
            result.append(self.unknown_label)
        else:
            best_pred = max(cur_predictions, key=cur_predictions.get)
            result.append(best_pred)
            del cur_predictions[best_pred]
    if unravel_result:
        predictions.append(result[0])
    else:
        predictions.append(result)